In [1]:
import keras as k
import numpy as np
import pandas as pd
from keras.layers import GRU, Dense, Activation, Embedding, Input, Dropout, LSTM
from keras.layers import Conv1D, MaxPooling1D, Flatten, GlobalMaxPooling1D, BatchNormalization
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer, one_hot
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
import tensorflow as tf

Using TensorFlow backend.


In [7]:
data = pd.read_csv('train.tsv',sep='\t')
test = pd.read_csv('test.tsv',sep='\t')

In [8]:
#Try with less words:
MAX_WORDS = 20000
EMBEDDING_DIM = 100

texts = data['Phrase'].values
tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(texts)
tokenizer.fit_on_texts(test['Phrase'].values)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

embeddings_index = {}
f = open('glove.6B.'+str(EMBEDDING_DIM)+'d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
#Pad the sequences to the max sequence length
MAX_SEQUENCE_LENGTH = max([len(i) for i in sequences])
x = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
y = np_utils.to_categorical(data['Sentiment'].values)

Found 17780 unique tokens.
Found 400000 word vectors.


In [ ]:
# BEST MODEL
UNITS = 64
model = Sequential()
model.add(Embedding(len(word_index) + 1,EMBEDDING_DIM,weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH,mask_zero=True))
model.add(LSTM(UNITS,recurrent_dropout=0.2,dropout=0.2))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.fit(x, y, batch_size=64, epochs=4, verbose=1, validation_split=0.2)

In [ ]:
h = model.fit(x, y, batch_size=64, epochs=1, verbose=1, validation_split=0.2)

In [ ]:
h.history['val_acc'][-1]

In [4]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform, conditional

def data():
    MAX_WORDS = 20000
    EMBEDDING_DIM = 100
    
    data = pandas.read_csv('train.tsv',sep='\t')
    test = pandas.read_csv('test.tsv',sep='\t')

    texts = data['Phrase'].values
    tokenizer = Tokenizer(num_words=MAX_WORDS)
    tokenizer.fit_on_texts(texts)
    tokenizer.fit_on_texts(test['Phrase'].values)
    sequences = tokenizer.texts_to_sequences(texts)

    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))

    embeddings_index = {}
    f = open('glove.6B.'+str(EMBEDDING_DIM)+'d.txt',encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        coefs = numpy.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    print('Found %s word vectors.' % len(embeddings_index))


    embedding_matrix = numpy.zeros((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

    #Pad the sequences to the max sequence length
    MAX_SEQUENCE_LENGTH = max([len(i) for i in sequences])
    x = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    y = np_utils.to_categorical(data['Sentiment'].values)
    return x, y, word_index, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH

def model(x, y, word_index, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH):
# BEST MODEL
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,EMBEDDING_DIM,weights=[embedding_matrix], 
                        input_length=MAX_SEQUENCE_LENGTH,mask_zero=True))
    
    model.add(LSTM({{choice([128,256,512])}},recurrent_dropout={{choice([0.2, 0.4])}},dropout={{choice([0.2, 0.4])}}))
    
    model.add(Dense({{choice([128,256,512])}},activation='relu'))
    
    model.add(Dropout({{choice([0.2,0.4])}}))
    
    model.add(Dense(5,activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer={{choice(['adam','rmsprop'])}}, metrics=['accuracy'])
    
    es = EarlyStopping(patience=2)
    
    history = model.fit(x, y, batch_size=128, epochs=5, verbose=2, validation_split=0.2,callbacks=[es])
    
    acc = max(history.history['val_acc'])
    
    print('Test accuracy:', acc)
    
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}




In [5]:
data_t = pd.read_csv('train.tsv',sep='\t')
test_t = pd.read_csv('test.tsv',sep='\t')

In [5]:

best_run, best_model = optim.minimize(model=model, data=data, algo=tpe.suggest, max_evals = 100, 
                                      trials=Trials(),notebook_name='Best',rseed=1)


print("Best performing model chosen hyper-parameters:")
print(best_run)

>>> Imports:
try:
    import keras
except:
    pass

try:
    import numpy
except:
    pass

try:
    import pandas
except:
    pass

try:
    from keras.layers import GRU, Dense, Activation, Embedding, Input, Dropout, LSTM
except:
    pass

try:
    from keras.layers import Conv1D, MaxPooling1D, Flatten, GlobalMaxPooling1D, BatchNormalization
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.preprocessing.sequence import pad_sequences
except:
    pass

try:
    from keras.preprocessing.text import Tokenizer, one_hot
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    from keras.callbacks import EarlyStopping
except:
    pass

try:
    import tensorflow
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform, conditional
except:
    pass

>>> Hyperas search s

Train on 124848 samples, validate on 31212 samples
Epoch 1/5
185s - loss: 1.0427 - acc: 0.5773 - val_loss: 0.9481 - val_acc: 0.6031
Epoch 2/5
183s - loss: 0.8958 - acc: 0.6303 - val_loss: 0.9206 - val_acc: 0.6162
Epoch 3/5
183s - loss: 0.8419 - acc: 0.6528 - val_loss: 0.9197 - val_acc: 0.6140
Epoch 4/5
183s - loss: 0.8083 - acc: 0.6678 - val_loss: 0.9129 - val_acc: 0.6221
Epoch 5/5
183s - loss: 0.7831 - acc: 0.6801 - val_loss: 0.9357 - val_acc: 0.6142
Test accuracy: 0.622132513449
Train on 124848 samples, validate on 31212 samples
Epoch 1/5
174s - loss: 1.0085 - acc: 0.5893 - val_loss: 0.9284 - val_acc: 0.6118
Epoch 2/5
172s - loss: 0.8322 - acc: 0.6534 - val_loss: 0.9182 - val_acc: 0.6195
Epoch 3/5
172s - loss: 0.7745 - acc: 0.6768 - val_loss: 0.9191 - val_acc: 0.6185
Epoch 4/5
172s - loss: 0.7368 - acc: 0.6901 - val_loss: 0.9276 - val_acc: 0.6187
Epoch 5/5
172s - loss: 0.7087 - acc: 0.7018 - val_loss: 0.9434 - val_acc: 0.6109
Test accuracy: 0.619505318605
Train on 124848 samples, val

Train on 124848 samples, validate on 31212 samples
Epoch 1/5
485s - loss: 0.9563 - acc: 0.6073 - val_loss: 0.9244 - val_acc: 0.6139
Epoch 2/5
486s - loss: 0.7920 - acc: 0.6705 - val_loss: 0.9189 - val_acc: 0.6198
Epoch 3/5
488s - loss: 0.7288 - acc: 0.6956 - val_loss: 0.9291 - val_acc: 0.6149
Epoch 4/5
485s - loss: 0.6860 - acc: 0.7114 - val_loss: 0.9469 - val_acc: 0.6195
Epoch 5/5
481s - loss: 0.6504 - acc: 0.7259 - val_loss: 0.9593 - val_acc: 0.6213
Test accuracy: 0.621299500208
Train on 124848 samples, validate on 31212 samples
Epoch 1/5
503s - loss: 1.0033 - acc: 0.5871 - val_loss: 1.0175 - val_acc: 0.5771
Epoch 2/5
498s - loss: 0.8722 - acc: 0.6396 - val_loss: 0.9182 - val_acc: 0.6167
Epoch 3/5
498s - loss: 0.8161 - acc: 0.6634 - val_loss: 0.9208 - val_acc: 0.6188
Epoch 4/5
498s - loss: 0.7806 - acc: 0.6792 - val_loss: 0.9425 - val_acc: 0.6131
Epoch 5/5
497s - loss: 0.7569 - acc: 0.6897 - val_loss: 0.9112 - val_acc: 0.6275
Test accuracy: 0.627547097163
Train on 124848 samples, val

663s - loss: 0.9828 - acc: 0.5972 - val_loss: 0.9248 - val_acc: 0.6130
Epoch 2/5
623s - loss: 0.8222 - acc: 0.6588 - val_loss: 0.9127 - val_acc: 0.6250
Epoch 3/5
630s - loss: 0.7592 - acc: 0.6835 - val_loss: 0.9229 - val_acc: 0.6239
Epoch 4/5
617s - loss: 0.7156 - acc: 0.6999 - val_loss: 0.9184 - val_acc: 0.6246
Epoch 5/5
624s - loss: 0.6830 - acc: 0.7107 - val_loss: 0.9365 - val_acc: 0.6184
Test accuracy: 0.625048058577
Train on 124848 samples, validate on 31212 samples
Epoch 1/5
529s - loss: 1.0268 - acc: 0.5809 - val_loss: 0.9620 - val_acc: 0.5987
Epoch 2/5
521s - loss: 0.8845 - acc: 0.6330 - val_loss: 0.9169 - val_acc: 0.6184
Epoch 3/5
518s - loss: 0.8301 - acc: 0.6588 - val_loss: 0.9385 - val_acc: 0.6048
Epoch 4/5
522s - loss: 0.7955 - acc: 0.6734 - val_loss: 0.9036 - val_acc: 0.6261
Epoch 5/5
522s - loss: 0.7711 - acc: 0.6843 - val_loss: 0.9180 - val_acc: 0.6171
Test accuracy: 0.626137382997
Train on 124848 samples, validate on 31212 samples
Epoch 1/5
274s - loss: 0.9456 - acc: 0

899s - loss: 0.7795 - acc: 0.6787 - val_loss: 0.9132 - val_acc: 0.6227
Epoch 5/5
901s - loss: 0.7542 - acc: 0.6916 - val_loss: 0.9103 - val_acc: 0.6262
Test accuracy: 0.626457772789
Train on 124848 samples, validate on 31212 samples
Epoch 1/5
977s - loss: 0.9587 - acc: 0.6065 - val_loss: 0.9202 - val_acc: 0.6129
Epoch 2/5
939s - loss: 0.7931 - acc: 0.6713 - val_loss: 0.9141 - val_acc: 0.6208
Epoch 3/5
929s - loss: 0.7293 - acc: 0.6937 - val_loss: 0.9309 - val_acc: 0.6202
Epoch 4/5
928s - loss: 0.6845 - acc: 0.7116 - val_loss: 0.9320 - val_acc: 0.6244
Epoch 5/5
926s - loss: 0.6494 - acc: 0.7246 - val_loss: 0.9593 - val_acc: 0.6170
Test accuracy: 0.62440727919
Train on 124848 samples, validate on 31212 samples
Epoch 1/5
940s - loss: 0.9729 - acc: 0.5993 - val_loss: 0.9412 - val_acc: 0.6082
Epoch 2/5
927s - loss: 0.8379 - acc: 0.6532 - val_loss: 0.9147 - val_acc: 0.6145
Epoch 3/5
928s - loss: 0.7793 - acc: 0.6778 - val_loss: 0.9078 - val_acc: 0.6268
Epoch 4/5
926s - loss: 0.7414 - acc: 0.

ResourceExhaustedError: OOM when allocating tensor with shape[128,49,2048]
	 [[Node: gradients_69/lstm_70/transpose_grad/transpose = Transpose[T=DT_FLOAT, Tperm=DT_INT32, _class=["loc:@lstm_70/transpose"], _device="/job:localhost/replica:0/task:0/gpu:0"](gradients_69/lstm_70/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3_grad/TensorArrayGatherV3, gradients_69/lstm_70/transpose_grad/InvertPermutation)]]

Caused by op 'gradients_69/lstm_70/transpose_grad/transpose', defined at:
  File "C:\Python\Python35\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Python\Python35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Python\Python35\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Python\Python35\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Python\Python35\lib\site-packages\ipykernel\kernelapp.py", l

In [24]:
{'recurrent_dropout': 0.3849339446151476, 'LSTM_1': 1, 'batch_size': 1, 'lstm_dropout': 0.0067245252545650724, 'dense_dropout_2': 0.4151995061036438, 'LSTM': 1, 'optimizer': 0}

156032/156060 [============================>.] - ETA: 0s

[0.73919014618455725, 0.69364987826035696]

In [11]:
#3 epoch val_acc: 0.629
bp629 = {'LSTM': 128, 'Dense': 256, 'optimizer': 'rmsprop', 'recurrent_dropout': 0.2, 'batch_size': 128, 'lstm_dropout': 0.2, 'dense_dropout': 0.2}
bm629 = best_model

In [10]:
best_model.fit(x, y, batch_size=128, epochs=4, verbose=1, validation_split=0.2)

Train on 124848 samples, validate on 31212 samples
Epoch 1/4
124848/124848 [==============================] - 223s - loss: 0.7863 - acc: 0.6754 - val_loss: 0.9202 - val_acc: 0.6241
Epoch 2/4
124848/124848 [==============================] - 217s - loss: 0.7506 - acc: 0.6917 - val_loss: 0.9140 - val_acc: 0.6231
Epoch 3/4
124848/124848 [==============================] - 217s - loss: 0.7249 - acc: 0.7015 - val_loss: 0.9201 - val_acc: 0.6290
Epoch 4/4
124848/124848 [==============================] - 219s - loss: 0.7040 - acc: 0.7116 - val_loss: 0.9268 - val_acc: 0.6242


In [ ]:
{'recurrent_dropout': 0.0, 'LSTM_1': 512, 'recurrent_dropout_1': 0.1, 'optimizer': 'adam', 'LSTM': 512, 'Dropout': 0.2}

In [8]:
best_model.fit(x, y, batch_size=128, epochs=4, verbose=1, validation_split=0.2)

Train on 124848 samples, validate on 31212 samples
Epoch 1/4
124848/124848 [==============================] - 772s - loss: 0.6935 - acc: 0.7082 - val_loss: 0.9363 - val_acc: 0.6204
Epoch 2/4
124848/124848 [==============================] - 767s - loss: 0.6453 - acc: 0.7277 - val_loss: 0.9444 - val_acc: 0.6255
Epoch 3/4
124848/124848 [==============================] - 770s - loss: 0.6051 - acc: 0.7419 - val_loss: 0.9762 - val_acc: 0.6217
Epoch 4/4
124848/124848 [==============================] - 756s - loss: 0.5673 - acc: 0.7548 - val_loss: 1.0209 - val_acc: 0.6183
